In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Div, Select, Button, ColorBar, CustomJS
from bokeh.layouts import row, column, layout
from bokeh.transform import cumsum, linear_cmap
from bokeh.palettes import Blues8, Spectral3
from bokeh.plotting import figure, output_file, show


output_notebook()

# Visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import folium 
from folium import plugins
plt.style.use("fivethirtyeight")


from plotly.offline import iplot
from plotly import tools
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

# Import the Files

In [ ]:
train_df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-3/train.csv")
test_df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-3/test.csv")

# See the first 5 data of Training dataset and testing dataset 

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# Check the Columns of the training Dataset**

In [ ]:
train_df.columns

# check the  datatypes of the training dataset columns

In [ ]:
train_df.dtypes

In [ ]:
# Importing Libraries for Plotting
import sys
import numpy as np
import pandas as pd

# Importing matplotlib
import matplotlib
import matplotlib.pyplot as plt

from matplotlib import style

#or use plt.show()
%matplotlib inline 

In [ ]:
country_df = train_df[train_df.Country_Region=='India'].groupby('Date')['ConfirmedCases','Fatalities'].sum()
country_df['day_count'] = list(range(1,len(country_df)+1))
ydata = country_df.ConfirmedCases
xdata = country_df.day_count
country_df['rate'] = (country_df.ConfirmedCases-country_df.ConfirmedCases.shift(1))/country_df.ConfirmedCases
country_df['increase'] = (country_df.ConfirmedCases-country_df.ConfirmedCases.shift(1))

plt.plot(xdata, ydata, '*')
plt.title("India")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
train_df.head()

In [ ]:
train_df["Date"] = pd.to_datetime(train_df["Date"])

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
#fill Na with the respective Country_Region
train_df['Province_State'].fillna(train_df['Country_Region'],inplace=True)

In [ ]:
#Check if the Null are filled with the same
train_df.isnull().sum()

In [ ]:
#correlation Matrix
train_df.corr()


In [ ]:
sns.heatmap(train_df.corr())

In [ ]:
confirmed_total_date = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
fatalities_total_date = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
total_date = confirmed_total_date.join(fatalities_total_date)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
total_date.plot(ax=ax1)
ax1.set_title("Global confirmed cases", size=13)
ax1.set_ylabel("Number of cases", size=13)
ax1.set_xlabel("Date", size=13)
fatalities_total_date.plot(ax=ax2, color='orange')
ax2.set_title("Global deceased cases", size=13)
ax2.set_ylabel("Number of cases", size=13)
ax2.set_xlabel("Date", size=13)

In [ ]:
#Graph which shows how the corona Virus got spread out in the whole world.


temp = train_df.groupby(['Date', 'Country_Region'])['ConfirmedCases'].sum().reset_index()
temp['Date'] = pd.to_datetime(temp['Date'])
temp['Date'] = temp['Date'].dt.strftime('%m/%d/%Y')
temp['size'] = temp['ConfirmedCases'].pow(0.3) * 1.5


fig = px.scatter_geo(temp, locations="Country_Region", locationmode='country names', 
                     color="ConfirmedCases", size='size', hover_name="Country_Region", 
                     range_color=[1,100],
                     projection="natural earth", animation_frame="Date", 
                     title='COVID-19: Cases Over Time', color_continuous_scale="Greens")
fig.show()

In [ ]:
#fatalies count country wise
train_df.groupby('Country_Region')['Fatalities','ConfirmedCases'].sum().reset_index()

In [ ]:
train_df.head()

In [ ]:
x = train_df[['Date']]
y1 = train_df[['ConfirmedCases']]
y2 = train_df[['Fatalities']]
x_test = test_df[['Date']]
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder
labelencoder_x=preprocessing.LabelEncoder()
x['Date']=labelencoder_x.fit_transform(x['Date'])

x_test['Date']=labelencoder_x.fit_transform(x_test['Date'])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
Tree_model = DecisionTreeClassifier(criterion='entropy')
##
Tree_model.fit(x,y1)
pred1 = Tree_model.predict(x_test)
pred1 = pd.DataFrame(pred1)
pred1.columns = ["ConfirmedCases_prediction"]
Tree_model.fit(x,y2)
pred2 = Tree_model.predict(x_test)
pred2 = pd.DataFrame(pred2)
pred2.columns = ["Death_prediction"]

In [ ]:
Sub = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-3/submission.csv")
Sub.columns
sub_new = Sub[["ForecastId"]]

In [ ]:
OP = pd.concat([pred1,pred2,sub_new],axis=1)
OP.head()
OP.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
OP = OP[['ForecastId','ConfirmedCases', 'Fatalities']]

In [ ]:
OP["ConfirmedCases"] = OP["ConfirmedCases"].astype(int)
OP["Fatalities"] = OP["Fatalities"].astype(int)

In [ ]:
## Using XGBoosting
import xgboost as xgb
model = xgb.XGBRegressor(learning_rate=0.01)
model.fit(x,y1)
pred_ra1 = model.predict(x_test)
pred_ra1 = pd.DataFrame(pred_ra1)
pred_ra1.columns = ["ConfirmedCases_prediction"]
model.fit(x,y2)
pred_ra2 = model.predict(x_test)
pred_ra2 = pd.DataFrame(pred_ra2)
pred_ra2.columns = ["Death_prediction"]
OP_ra = pd.concat([sub_new,pred_ra1,pred_ra2],axis=1)
OP_ra.head()
OP_ra.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']
OP_ra["ConfirmedCases"] = OP_ra["ConfirmedCases"].astype(int)
OP_ra["Fatalities"] = OP_ra["Fatalities"].astype(int)
OP_ra.head()

In [ ]:
OP_ra.to_csv("submission.csv",index=False)